In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h2 style="color:blue" align="left"> 1. Import necessary Libraries </h2>

In [ ]:
# Read data
import numpy as np                           # Linear Algebra (calculate the mean and standard deviation)
import pandas as pd                          # manipulate data, data processing, load csv file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot as plt              # Visualization using matplotlib
%matplotlib inline
import seaborn as sns                        # Visualization using seaborn

# style
plt.style.use("fivethirtyeight")             # Set Graphs Background style using matplotlib
sns.set_style("darkgrid")                    # Set Graphs Background style using seaborn

import warnings                              # To ignore any warnings
warnings.filterwarnings("ignore")

<h2 style=color:blue align="left"> 2. EDA(Exploratory Data Analysis) </h2>

<h2 style=color:green align="left"> a. Understand the data </h2>

In [ ]:
from sklearn.model_selection import train_test_split                     # split  data into training and testing sets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier                           # Decision tree Regression
import xgboost
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, classification_report      # this creates a confusion matrix
from sklearn.model_selection import GridSearchCV                         # this will do cross validation

In [ ]:
# Loading the train, test & sample dataset
train = pd.read_csv("/kaggle/input/health-insurance-lead-prediction/train.csv")
test = pd.read_csv("/kaggle/input/health-insurance-lead-prediction/test.csv")

In [ ]:
# make copy of train & test
train_original = train.copy()
test_original = test.copy()

In [ ]:
# checking dimension (num of rows and columns) of datasets
display(train.shape)
display(test.shape)

In [ ]:
# import first rows
train.head()

In [ ]:
# import first rows
test.head()

In [ ]:
# check dataframe structure like columns and its counts, datatypes & Null Values
train.info()

In [ ]:
# check dataframe structure like columns and its counts, datatypes & Null Values
test.info()

In [ ]:
# Gives number of data points in each variable
train.count()

In [ ]:
# descriptive statistics
train.describe()

### 1. City_Code

In [ ]:
train["City_Code"].nunique()

In [ ]:
test["City_Code"].nunique()

In [ ]:
plt.figure(figsize=(10,8))
train["City_Code"].value_counts().plot.bar()

### 2. Accomodation_Type

In [ ]:
train["Accomodation_Type"].nunique()

In [ ]:
test["Accomodation_Type"].nunique()

In [ ]:
train["Accomodation_Type"].value_counts()

### 3. Reco_Insurance_Type

In [ ]:
train["Reco_Insurance_Type"].nunique()

In [ ]:
test["Reco_Insurance_Type"].nunique()

In [ ]:
train["Reco_Insurance_Type"].value_counts()

### 4. Is_Spouse

In [ ]:
train["Is_Spouse"].nunique()

In [ ]:
test["Is_Spouse"].nunique()

In [ ]:
train["Is_Spouse"].value_counts()

### 5. Health Indicator

In [ ]:
train["Health Indicator"].nunique()

In [ ]:
test["Health Indicator"].nunique()

In [ ]:
train["Health Indicator"].value_counts()

### 6. Holding_Policy_Duration

In [ ]:
train["Holding_Policy_Duration"].nunique()

In [ ]:
train["Holding_Policy_Duration"].value_counts()

In [ ]:
train["Holding_Policy_Duration"] = train["Holding_Policy_Duration"].replace('14+', 14)
test["Holding_Policy_Duration"] = test["Holding_Policy_Duration"].replace('14+', 14)

In [ ]:
train["Holding_Policy_Duration"].value_counts()

### 7. Holding_Policy_Type

In [ ]:
train["Holding_Policy_Type"].nunique()

In [ ]:
test["Holding_Policy_Type"].nunique()

In [ ]:
train["Holding_Policy_Type"].value_counts()

### 8. Response

In [ ]:
train["Response"].nunique()

In [ ]:
train["Response"].value_counts()

<h2 style=color:green align="left"> b. Clean the data </h2>

### i) Drop unwanted features

In [ ]:
train.drop(['ID', 'City_Code', 'Region_Code'], axis=1, inplace=True)

test.drop(['ID', 'City_Code', 'Region_Code'], axis=1, inplace=True)

In [ ]:
### ii) Missing Values

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(train.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(test.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
train["Holding_Policy_Duration"] = train["Holding_Policy_Duration"].astype('float64')
test["Holding_Policy_Duration"] = test["Holding_Policy_Duration"].astype('float64')

In [ ]:
train["Health Indicator"] = train["Health Indicator"].fillna(train["Health Indicator"].mode()[0])
train["Holding_Policy_Duration"] = train["Holding_Policy_Duration"].fillna(train["Holding_Policy_Duration"].median())
train["Holding_Policy_Type"] = train["Holding_Policy_Type"].fillna(train["Holding_Policy_Type"].median())

test["Health Indicator"] = test["Health Indicator"].fillna(test["Health Indicator"].mode()[0])
test["Holding_Policy_Duration"] = test["Holding_Policy_Duration"].fillna(test["Holding_Policy_Duration"].median())
test["Holding_Policy_Type"] = test["Holding_Policy_Type"].fillna(test["Holding_Policy_Type"].median())

In [ ]:
display(train.head(3))
display(test.head(3))

<h2 style=color:green align="left"> c. Find Outliers </h2>

In [ ]:
plt.figure(figsize=(17,10))
sns.boxplot(data=train)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.show()

In [ ]:
plt.figure(figsize=(17,10))
sns.boxplot(data=test)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.show()

<h2 style=color:green align="left"> d. Skew and Kurtosis </h2>

In [ ]:
display(train.skew())
display(test.skew())

In [ ]:
sns.distplot(train['Reco_Policy_Premium'])
Skew_Reco_Policy_Premium = train['Reco_Policy_Premium'].skew()
plt.title("Skew:"+str(Skew_Reco_Policy_Premium))

In [ ]:
sns.distplot(test['Reco_Policy_Premium'])
Skew_Reco_Policy_Premium = test['Reco_Policy_Premium'].skew()
plt.title("Skew:"+str(Skew_Reco_Policy_Premium))

In [ ]:
sns.distplot(train['Holding_Policy_Duration'])
Skew_Holding_Policy_Duration = train['Holding_Policy_Duration'].skew()
plt.title("Skew:"+str(Skew_Holding_Policy_Duration))

In [ ]:
sns.distplot(test['Holding_Policy_Duration'])
Skew_Holding_Policy_Duration = test['Holding_Policy_Duration'].skew()
plt.title("Skew:"+str(Skew_Holding_Policy_Duration))

In [ ]:
sns.distplot(train['Holding_Policy_Type'])
Skew_Holding_Policy_Type = train['Holding_Policy_Type'].skew()
plt.title("Skew:"+str(Skew_Holding_Policy_Type))

In [ ]:
sns.distplot(train['Reco_Policy_Cat'])
Skew_Reco_Policy_Cat = train['Reco_Policy_Cat'].skew()
plt.title("Skew:"+str(Skew_Reco_Policy_Cat))

<h2 style=color:green align="left"> e. Analysis of Relationship between variables </h2>

In [ ]:
train.corr()['Reco_Policy_Cat']

### ii) Heatmap

In [ ]:
plt.figure(figsize=(10,6)) 
sns.heatmap(train.corr(), annot=True, linecolor = 'white', linewidths = 1, cmap="YlGnBu")
plt.show()

<h2 style=color:green align="left"> f. Data Visualisation </h2>

### i) Univariate Analysis

In [ ]:
numeric_cols_train = train.select_dtypes(include=[np.number])
display(numeric_cols_train.head())

In [ ]:
numeric_cols_test = test.select_dtypes(include=[np.number])
display(numeric_cols_test.head())

In [ ]:
plt.figure(figsize = (15,10))
feature = numeric_cols_train.columns[0:6]
for i in enumerate(feature):
    plt.subplot(3,2, i[0]+1)
    sns.distplot(numeric_cols_train[i[1]], color='crimson')

In [ ]:
plt.figure(figsize = (15,10))
feature = numeric_cols_test.columns[0:6]
for i in enumerate(feature):
    plt.subplot(3,2, i[0]+1)
    sns.distplot(numeric_cols_test[i[1]], color='crimson')

In [ ]:
plt.figure(figsize = (13, 9)) 
train.boxplot()
plt.show()

### ii) Bivariate Analysis

In [ ]:
# Setting color palette
colors = ['#D32F2F','#1976D2']
sns.palplot(sns.color_palette(colors))

In [ ]:
plt.figure(figsize=(12,10))
sns.scatterplot(x="Holding_Policy_Duration", y="Holding_Policy_Type", hue="Accomodation_Type", data=train, palette=colors, marker="X")
plt.show()

In [ ]:
plt.figure(figsize=(9,7))
sns.boxplot(data=train, x='Accomodation_Type', y='Holding_Policy_Type')
plt.show()

### iii) Multivariate Analysis

In [ ]:
# Pairplot
sns.pairplot(train, hue='Accomodation_Type')
plt.show()

<h2 style="color:blue" align="left"> 3. Model building and Evaluation </h2>

In [ ]:
# Transform discrete values to columns with 1 and 0s
train_OHE = pd.get_dummies(train)

# Do the same for competition data
test_OHE = pd.get_dummies(test)

In [ ]:
display(train_OHE.head())
display(test_OHE.head())

In [ ]:
print("Training Data Shape (Rows,Columns):",train_OHE.shape)
print("Competition Data Shape (Rows,Columns):", test_OHE.shape)

In [ ]:
# Independant variable
X = train_OHE.drop('Response',axis=1)        # All rows & columns exclude Target features

# Dependant variable
y = train_OHE['Response']                   # Only target feature

In [ ]:
# split  data into training and testing sets of 80:20 ratio
# 20% of test size selected
# random_state is random seed
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=4)

In [ ]:
# shape of X & Y test / train
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## 1. Logistic Regression

In [ ]:
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

In [ ]:
y_pred_LogReg = LogReg.predict(X_test)

In [ ]:
print("Train Score {:.2f} & Test Score {:.2f}".format(LogReg.score(X_train, y_train), LogReg.score(X_test, y_test)))

## 2. Random Forest Classifier

In [ ]:
rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=1)
rf.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf.predict(X_test)

In [ ]:
print("Train Score {:.2f} & Test Score {:.2f}".format(rf.score(X_train, y_train), rf.score(X_test, y_test)))

## 3. XGBoost

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

In [ ]:
y_pred_xgb = xgb.predict(X_test)

In [ ]:
print("Train Score {:.2f} & Test Score {:.2f}".format(xgb.score(X_train, y_train), xgb.score(X_test, y_test)))

## 4. CATBOOST

In [ ]:
CB = CatBoostClassifier(verbose=0, n_estimators=100)
CB.fit(X_train, y_train)

In [ ]:
y_pred_xgb = CB.predict(X_test)

In [ ]:
print("Train Score {:.2f} & Test Score {:.2f}".format(CB.score(X_train, y_train), CB.score(X_test, y_test)))

## 5. LGBM

In [ ]:
lgbm_model = LGBMClassifier()
lgbm_model = lgbm_model.fit(X_train, y_train)

In [ ]:
y_pred_LGBM = lgbm_model.predict(X_test)

In [ ]:
print("Train Score {:.2f} & Test Score {:.2f}".format(lgbm_model.score(X_train,y_train),lgbm_model.score(X_test,y_test)))

In [ ]:
y_pred_test = xgb.predict(test_OHE)

## Submission

In [ ]:
submission = pd.DataFrame({'ID': test_original['ID'], 'Response': y_pred_test})
submission.to_csv('Insurance.csv', index=False)